# Assignment 2

conducted by:

- Mirco Lescart

- Freddy Fernandes

- Arina Sadeghi Khiabanian

- Parsa Mastouri Kashani

Installing the transformers library and additional libraries:

In [1]:
!pip install -q transformers
!pip install mlcm

!pip install datasets==2.13.2
!pip install accelerate -U
!pip install evaluate
!pip install torchsummary

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 512.7/512.7 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.0 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.9/270.9 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 2.6 MB/s eta 0:00:00


Importing stock ml libraries

In [2]:
import numpy as np
import pandas as pd
from sklearn import metrics
import transformers
import torch
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler
from transformers import AutoModel, AutoTokenizer, get_linear_schedule_with_warmup
from sklearn.metrics import classification_report, accuracy_score, f1_score
from transformers import BertTokenizer, BertModel, BertConfig

Setting up the device for GPU usage if available

In [3]:
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'
print(f'device: {device}')

device: cuda


# Task 1

Importing the data and making them into pandas dataframes:

In [4]:
args_train = pd.read_csv("arguments-training.tsv", delimiter='\t')
labels_train = pd.read_csv("labels-training.tsv", delimiter='\t')

args_validation = pd.read_csv("arguments-validation.tsv", delimiter='\t')
labels_validation = pd.read_csv("labels-validation.tsv", delimiter='\t')

args_test = pd.read_csv("arguments-test.tsv", delimiter='\t')
labels_test = pd.read_csv("labels-test.tsv", delimiter='\t')

Implementing the necessary logic to get level 3 category features:

In [5]:
def concat_level3(df):

  columns_to_include = ['Self-direction: thought',
                        'Self-direction: action',
                        'Stimulation',
                        'Hedonism']
  df['Openness to change'] = df[columns_to_include].apply(lambda row: any(row == 1), axis=1).astype(int)

  columns_to_include = ['Hedonism',
                        'Achievement',
                        'Stimulation',
                        'Power: dominance',
                        'Power: resources',
                        'Face']
  df['Self-enhancement'] = df[columns_to_include].apply(lambda row: any(row == 1), axis=1).astype(int)

  columns_to_include = ['Face',
                      'Security: personal',
                      'Security: societal',
                      'Security: societal',
                      'Tradition',
                      'Conformity: rules',
                      'Conformity: interpersonal',
                      'Humility']
  df['Conservation'] = df[columns_to_include].apply(lambda row: any(row == 1), axis=1).astype(int)

  columns_to_include = ['Humility',
                        'Benevolence: caring',
                        'Benevolence: dependability',
                        'Universalism: concern',
                        'Universalism: nature',
                        'Universalism: tolerance',
                        'Universalism: objectivity']
  df['Self-transcendence'] = df[columns_to_include].apply(lambda row: any(row == 1), axis=1).astype(int)

  return df.iloc[:, -4:]

In [6]:
labels_train = concat_level3(labels_train)
labels_validation = concat_level3(labels_validation)
labels_test = concat_level3(labels_test)

Concatenating the labels and the arguments together

In [7]:
# Concatenate along columns (axis=1)
train_set = pd.concat([args_train, labels_train], axis=1)
validation_set = pd.concat([args_validation, labels_validation], axis=1)
test_set = pd.concat([args_test, labels_test], axis=1)

We drop Argument ID because it's irrelevant in our problem

In [8]:
train_set = train_set.drop('Argument ID', axis=1)
validation_set = validation_set.drop('Argument ID', axis=1)
test_set = test_set.drop('Argument ID', axis=1)

We turn 'in favor of' and 'against' into 0 and 1 because it's easier to feed the model directly and it also takes less tokens to tokenize.

In [9]:
train_set['Stance'] = train_set['Stance'].replace({'in favor of': 1, 'against': 0})
validation_set['Stance'] = validation_set['Stance'].replace({'in favor of': 1, 'against': 0})
test_set['Stance'] = test_set['Stance'].replace({'in favor of': 1, 'against': 0})

In [10]:
train_set.head()

,Conclusion,Stance,Premise,Openness to change,Self-enhancement,Conservation,Self-transcendence
0,We should ban human cloning,1,we should ban human cloning as it will only ca...,0,0,1,0
1,We should ban fast food,1,fast food should be banned because it is reall...,0,0,1,0
2,We should end the use of economic sanctions,0,sometimes economic sanctions are the only thin...,0,1,1,0
3,We should abolish capital punishment,0,capital punishment is sometimes the only optio...,0,0,1,1
4,We should ban factory farming,0,factory farming allows for the production of c...,0,0,1,1


# Task 2

# Baseline Model

In [11]:
from sklearn.dummy import DummyClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score

In [12]:
from sklearn.metrics import precision_recall_fscore_support

num_labels = 4
classifiers = []
f1_scores = []
accuracies = []

for label in range(num_labels):

    # Create a random uniform classifier for the current label
    random_classifier = DummyClassifier(strategy='uniform')

    random_classifier.fit(train_set.iloc[:, :3], train_set.iloc[:, label+3])

    # Predict
    predictions = random_classifier.predict(test_set.iloc[:, :3])

    classifiers.append(random_classifier)

    # Calculate accuracy for the current label
    accuracy = accuracy_score(test_set.iloc[:, label+3], predictions)
    accuracies.append(accuracy)

    # Calculate F1 score for the current label
    f1 = f1_score(test_set.iloc[:, label+3], predictions)
    f1_scores.append(f1)

    print(f'{train_set.columns[3+label]}: ')

    print(classification_report(test_set.iloc[:, label+3], predictions))

    print(f"Label {label}: Accuracy = {accuracy:.4f}, F1 score = {f1:.4f},   {train_set.columns[3+label]}")

print(f"Overall F1 score: {sum(f1_scores)/len(f1_scores):.4f}")

Openness to change: 
              precision    recall  f1-score   support

           0       0.69      0.50      0.58      1102
           1       0.29      0.47      0.36       474

    accuracy                           0.49      1576
   macro avg       0.49      0.49      0.47      1576
weighted avg       0.57      0.49      0.51      1576

Label 0: Accuracy = 0.4918, F1 score = 0.3566,   Openness to change
Self-enhancement: 
              precision    recall  f1-score   support

           0       0.56      0.54      0.55       892
           1       0.43      0.46      0.45       684

    accuracy                           0.50      1576
   macro avg       0.50      0.50      0.50      1576
weighted avg       0.51      0.50      0.50      1576

Label 1: Accuracy = 0.5032, F1 score = 0.4451,   Self-enhancement
Conservation: 
              precision    recall  f1-score   support

           0       0.29      0.47      0.36       457
           1       0.71      0.52      0.60     

In [13]:
from sklearn.metrics import precision_recall_fscore_support

num_labels = 4
classifiers = []
f1_scores = []
accuracies = []

for label in range(num_labels):

    # Create a random uniform classifier for the current label
    random_classifier = DummyClassifier(strategy='most_frequent')

    random_classifier.fit(train_set.iloc[:, :3], train_set.iloc[:, label+3])

    # Predict
    predictions = random_classifier.predict(test_set.iloc[:, :3])

    classifiers.append(random_classifier)

    # Calculate accuracy for the current label
    accuracy = accuracy_score(test_set.iloc[:, label+3], predictions)
    accuracies.append(accuracy)

    # Calculate F1 score for the current label
    f1 = f1_score(test_set.iloc[:, label+3], predictions, average='micro')
    f1_scores.append(f1)

    print(f'{train_set.columns[3+label]}: ')

    print(classification_report(test_set.iloc[:, label+3], predictions, zero_division=1))

    print(f"Label {label}: Accuracy = {accuracy:.4f}, F1 score = {f1:.4f},   {train_set.columns[3+label]}")

print(f"Overall F1 score: {sum(f1_scores)/len(f1_scores):.4f}")

Openness to change: 
              precision    recall  f1-score   support

           0       0.70      1.00      0.82      1102
           1       1.00      0.00      0.00       474

    accuracy                           0.70      1576
   macro avg       0.85      0.50      0.41      1576
weighted avg       0.79      0.70      0.58      1576

Label 0: Accuracy = 0.6992, F1 score = 0.6992,   Openness to change
Self-enhancement: 
              precision    recall  f1-score   support

           0       0.57      1.00      0.72       892
           1       1.00      0.00      0.00       684

    accuracy                           0.57      1576
   macro avg       0.78      0.50      0.36      1576
weighted avg       0.75      0.57      0.41      1576

Label 1: Accuracy = 0.5660, F1 score = 0.5660,   Self-enhancement
Conservation: 
              precision    recall  f1-score   support

           0       1.00      0.00      0.00       457
           1       0.71      1.00      0.83     

### Hyperparameters

In [14]:
# Sections of config
# bert-base-uncased
# prajjwal1/bert-tiny
# distilbert-base-uncased
# roberta-base
card_name = 'roberta-base'
labels = ['Openness to change', 'Conservation', 'Self-enhancement', 'Self-transcendence']

# Defining some key variables that will be used later on in the training
MAX_LEN = 250
TRAIN_BATCH_SIZE = 8
VALID_BATCH_SIZE = 8
EPOCHS = 3
LEARNING_RATE = 2e-05
tokenizer = AutoTokenizer.from_pretrained(card_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [15]:
class CustomDataset(Dataset):

    def __init__(self, dataframe, tokenizer, max_len):
        self.data = dataframe
        self.data['list'] = dataframe[dataframe.columns[3:]].values.tolist()
        self.tokenizer = tokenizer
        self.Conclusion = dataframe.Conclusion
        self.Premise = dataframe.Premise
        self.Stance = dataframe.Stance
        self.targets = self.data.list
        self.max_len = max_len

    def __len__(self):
        return len(self.Stance)

    def __getitem__(self, index):

        Stance = self.Stance[index]

        Conclusion = str(self.Conclusion[index])
        Conclusion = " ".join(Conclusion.split())

        Premise = str(self.Premise[index])
        Premise = " ".join(Premise.split())

        CP_text = f"|Conclusion| {Conclusion} |Premise| {Premise}"
        CPS_text = f"|Stance| {Stance} |Conclusion| {Conclusion} |Premise| {Premise}"

        C_inputs = self.tokenizer.encode_plus(
            Conclusion,
            None,
            add_special_tokens=True,
            max_length=self.max_len,
            truncation=True,
            padding='max_length',
            return_token_type_ids=True
        )
        C_ids = C_inputs['input_ids']
        C_mask = C_inputs['attention_mask']
        C_token_type_ids = C_inputs["token_type_ids"]

        Premise_inputs = self.tokenizer.encode_plus(
            Premise,
            None,
            add_special_tokens=True,
            max_length=self.max_len,
            truncation=True,
            padding='max_length',
            return_token_type_ids=True
        )
        Premise_ids = Premise_inputs['input_ids']
        Premise_mask = Premise_inputs['attention_mask']
        Premise_token_type_ids = Premise_inputs["token_type_ids"]

        CP_inputs = self.tokenizer.encode_plus(
            Premise,
            None,
            add_special_tokens=True,
            max_length=self.max_len,
            truncation=True,
            padding='max_length',
            return_token_type_ids=True
        )
        CP_ids = CP_inputs['input_ids']
        CP_mask = CP_inputs['attention_mask']
        CP_token_type_ids = CP_inputs["token_type_ids"]

        CPS_inputs = self.tokenizer.encode_plus(
            CPS_text,
            None,
            add_special_tokens=True,
            max_length=self.max_len,
            truncation=True,
            padding='max_length',
            return_token_type_ids=True
        )

        CPS_ids = CPS_inputs['input_ids']
        CPS_mask = CPS_inputs['attention_mask']
        CPS_token_type_ids = CPS_inputs["token_type_ids"]

        return {
            'C_ids': torch.tensor(C_ids, dtype=torch.long),
            'C_mask': torch.tensor(C_mask, dtype=torch.long),
            'C_token_type_ids': torch.tensor(C_token_type_ids, dtype=torch.long),

            'Premise_ids': torch.tensor(Premise_ids, dtype=torch.long),
            'Premise_mask': torch.tensor(Premise_mask, dtype=torch.long),
            'Premise_token_type_ids': torch.tensor(Premise_token_type_ids, dtype=torch.long),

            'CP_ids': torch.tensor(CP_ids, dtype=torch.long),
            'CP_mask': torch.tensor(CP_mask, dtype=torch.long),
            'CP_token_type_ids': torch.tensor(CP_token_type_ids, dtype=torch.long),

            'CPS_ids': torch.tensor(CPS_ids, dtype=torch.long),
            'CPS_mask': torch.tensor(CPS_mask, dtype=torch.long),
            'CPS_token_type_ids': torch.tensor(CPS_token_type_ids, dtype=torch.long),

            'Stance': torch.tensor(self.Stance[index], dtype=torch.float),

            'targets': torch.tensor(self.targets[index], dtype=torch.float),
        }

In [16]:
train_dataset = CustomDataset(train_set, tokenizer, MAX_LEN)
validation_dataset = CustomDataset(validation_set, tokenizer, MAX_LEN)
test_dataset = CustomDataset(test_set, tokenizer, MAX_LEN)

In [17]:
print(f"Shape of the training dataset: {len(train_dataset)}")
print(f"Shape of the validation dataset: {len(validation_dataset)}")
print(f"Shape of the test dataset: {len(test_dataset)}")

Shape of the training dataset: 5393
Shape of the validation dataset: 1896
Shape of the test dataset: 1576


In [18]:
train_params = {'batch_size': TRAIN_BATCH_SIZE,
                'shuffle': False,
                'num_workers': 0
                }

validation_params = {'batch_size': VALID_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

test_params = {'batch_size': VALID_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

training_loader = DataLoader(train_dataset, **train_params)
validation_loader = DataLoader(validation_dataset, **validation_params)
testing_loader = DataLoader(test_dataset, **test_params)

In [19]:
# Creating the customized model, by adding a drop out and a dense layer on top of distil bert to get the final output for the model.
import torch.nn.functional as F


class BERTClass(torch.nn.Module):
    def __init__(self, card_name):
        super(BERTClass, self).__init__()
        self.transformer = AutoModel.from_pretrained(card_name, return_dict=False)
        self.dropout = torch.nn.Dropout(0.3)
        self.dense1 = torch.nn.Linear(768, 384)
        self.dense2 = torch.nn.Linear(384, 192)
        self.dense3 = torch.nn.Linear(192, 4)

    def forward(self, ids, mask, token_type_ids):
        _, output = self.transformer(ids, attention_mask = mask, token_type_ids = token_type_ids, return_dict=False)
        output = self.dropout(output)
        output = self.dense1(output)
        output = F.relu(output)
        output = self.dense2(output)
        output = F.relu(output)
        output = self.dense3(output)
        return output

C_model = BERTClass(card_name)
C_model.to(device)

CP_model = BERTClass(card_name)
CP_model.to(device)

CPS_model = BERTClass(card_name)
CPS_model.to(device)

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BERTClass(
  (transformer): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): Layer

This is a model we came up with just for experimentation which gives the premise to one transformer and the conclusion to another one and then gets 768 features out of each of the transformers, then concatenates the features including the stance (just one feature) and feeds it to a dense layer (1537 by 100) which gets followed by two other dense layers.

In [31]:
import torch.nn.functional as F

class UltimaClass(torch.nn.Module):
    def __init__(self, card_name):
        super(UltimaClass, self).__init__()
        self.l1 = AutoModel.from_pretrained(card_name, return_dict=False)
        self.l4 = AutoModel.from_pretrained(card_name, return_dict=False)
        self.l2 = torch.nn.Dropout(0.3)
        self.l3 = torch.nn.Linear(1537, 100)
        self.l5 = torch.nn.Linear(100, 25)
        self.l6 = torch.nn.Linear(25, 4)

    def forward(self,
                Stance,
                Conclusion_ids,
                Conclusion_mask,
                Conclusion_token_type_ids,
                Premise_ids,
                Premise_mask,
                Premise_token_type_ids):

        _, output_Conclusion= self.l1(Conclusion_ids,
                             Conclusion_mask,
                             Conclusion_token_type_ids,
                             return_dict=False)


        _, output_Premise= self.l4(Premise_ids,
                             Premise_mask,
                             Premise_token_type_ids,
                             return_dict=False)


        output_Conclusion_wdrop = self.l2(output_Conclusion)
        output_Premise_wdrop = self.l2(output_Premise)

        # Assuming you want to concatenate along dimension 1
        Stance = Stance.unsqueeze(1)

        concatenated_input = torch.cat((Stance,
                                        output_Conclusion_wdrop,
                                        output_Premise_wdrop),
                                       dim=1)

        output = F.relu(self.l3(concatenated_input))
        output = F.relu(self.l5(output))
        output = self.l6(output)
        return output

ultima_model = UltimaClass(card_name)
ultima_model.to(device)

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


UltimaClass(
  (l1): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((7

Here we define the loss function which is binary cross entropy with logits as our problem is multi-label classification and we don't use sigmoid layers before we get our output (BCEWithLogitsLoss is more stable than normal BCE Loss)

In [20]:
def loss_fn(outputs, targets):
    return torch.nn.BCEWithLogitsLoss()(outputs, targets)

The training loops:

In [29]:
def trainBert(model, dataloader, optimizer, scheduler, LEARNING_RATE, EPOCHS, epoch, name):
    size = len(dataloader.dataset)
    model.train()
    for batch, data in enumerate(dataloader, 0):
        ids = data[f'{name}_ids'].to(device, dtype = torch.long)
        mask = data[f'{name}_mask'].to(device, dtype = torch.long)
        token_type_ids = data[f'{name}_token_type_ids'].to(device, dtype = torch.long)
        targets = data['targets'].to(device, dtype = torch.float)

        optimizer.zero_grad()

        outputs = model(ids, mask, token_type_ids)

        loss = loss_fn(outputs, targets)

        loss.backward()
        optimizer.step()
        scheduler.step()

        if batch%5000==0:
            print(f'Epoch: {epoch}, Loss:  {loss.item()}')
            print()
            # print(f"outputs: {outputs}")
            # print(f"targets: {targets}")

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(ids)
            print(f"Train loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


In [28]:
def ultima_train(model, dataloader, optimizer, scheduler, LEARNING_RATE, EPOCHS, epoch):
    size = len(dataloader.dataset)
    model.train()
    for batch, data in enumerate(dataloader, 0):

        Stance = data['Stance'].to(device, dtype = torch.long)

        conclusion_mask = data['C_mask'].to(device, dtype = torch.long)
        conclusion_ids = data['C_ids'].to(device, dtype = torch.long)
        conclusion_token_type_ids = data['C_token_type_ids'].to(device, dtype = torch.long)

        premise_ids = data['Premise_ids'].to(device, dtype = torch.long)
        premise_mask = data['Premise_mask'].to(device, dtype = torch.long)
        premise_token_type_ids = data['Premise_token_type_ids'].to(device, dtype = torch.long)

        targets = data['targets'].to(device, dtype = torch.float)

        optimizer.zero_grad()

        outputs = model(Stance,
                conclusion_ids,
                conclusion_mask,
                conclusion_token_type_ids,
                premise_ids,
                premise_mask,
                premise_token_type_ids)

        loss = loss_fn(outputs, targets)

        loss.backward()
        optimizer.step()
        scheduler.step()

        if batch%5000==0:
            print(f'Epoch: {epoch}, Loss:  {loss.item()}')
            print()
            # print(f"outputs: {outputs}")
            # print(f"targets: {targets}")

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(Stance)
            print(f"Train loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

The validation loops:

In [31]:
def validation(epoch, val_loss_min_input, model, dataloader, name):
    num_batches = len(dataloader)
    model.eval()
    val_loss = 0
    with torch.no_grad():
        for batch_idx, data in enumerate(dataloader, 0):
            ids = data[f'{name}_ids'].to(device, dtype = torch.long)
            mask = data[f'{name}_mask'].to(device, dtype = torch.long)
            token_type_ids = data[f'{name}_token_type_ids'].to(device, dtype = torch.long)
            targets = data['targets'].to(device, dtype = torch.float)
            outputs = model(ids, mask, token_type_ids)
            val_loss += loss_fn(outputs, targets).item()

        val_loss /= num_batches
        print(val_loss)
        #outputs, targets = fin_outputs, fin_targets
        print(f"\nValidation loss: {val_loss:>8f}.")
        if val_loss <= val_loss_min_input:
            #create checkpoint variable and add important data
            if epoch > 0:
                print('Validation loss decreased ({:.8f} --> {:.8f}).  Saving model ...'.format(val_loss_min_input, val_loss))
            else: print('Saving model ...')
            # save best model
            torch.save(model.state_dict(), f"{name}_model.pth")
            print("Saved PyTorch Model State to model.pth\n")
            val_loss_min_input = val_loss
        else:
          print('validation loss didn\'t go down -____-')
    return val_loss_min_input

In [30]:
def ultima_validation(epoch, val_loss_min_input, model, dataloader, name):
    num_batches = len(dataloader)
    model.eval()
    val_loss = 0
    with torch.no_grad():
        for batch_idx, data in enumerate(dataloader, 0):
            Stance = data['Stance'].to(device, dtype = torch.long)

            conclusion_ids = data['C_ids'].to(device, dtype = torch.long)
            conclusion_mask = data['C_mask'].to(device, dtype = torch.long)
            conclusion_token_type_ids = data['C_token_type_ids'].to(device, dtype = torch.long)

            premise_ids = data['Premise_ids'].to(device, dtype = torch.long)
            premise_mask = data['Premise_mask'].to(device, dtype = torch.long)
            premise_token_type_ids = data['Premise_token_type_ids'].to(device, dtype = torch.long)

            targets = data['targets'].to(device, dtype = torch.float)

            outputs = model(Stance,
                conclusion_ids,
                conclusion_mask,
                conclusion_token_type_ids,
                premise_ids,
                premise_mask,
                premise_token_type_ids)

            val_loss += loss_fn(outputs, targets).item()

        val_loss /= num_batches
        print(val_loss)
        #outputs, targets = fin_outputs, fin_targets
        print(f"\nValidation loss: {val_loss:>8f}.")
        if val_loss <= val_loss_min_input:
            #create checkpoint variable and add important data
            if epoch > 0:
                print('Validation loss decreased ({:.8f} --> {:.8f}).  Saving model ...'.format(val_loss_min_input, val_loss))
            else: print('Saving model ...')
            # save best model
            torch.save(model.state_dict(), f"{name}_model.pth")
            print("Saved PyTorch Model State to model.pth\n")
            val_loss_min_input = val_loss
        else:
          print('validation loss didn\'t go down -____-')
    return val_loss_min_input

The test loop:

In [33]:
def test(model, data_loader, name):
    model.eval()
    fin_targets=[]
    fin_outputs=[]
    with torch.no_grad():
        for _, data in enumerate(data_loader, 0):
            ids = data[f'{name}_ids'].to(device, dtype = torch.long)
            mask = data[f'{name}_mask'].to(device, dtype = torch.long)
            token_type_ids = data[f'{name}_token_type_ids'].to(device, dtype = torch.long)
            targets = data['targets'].to(device, dtype = torch.float)
            outputs = model(ids, mask, token_type_ids)
            fin_targets.extend(targets.cpu().detach().numpy().tolist())
            fin_outputs.extend(torch.sigmoid(outputs).cpu().detach().numpy().tolist())
    return fin_outputs, fin_targets

In [35]:
def ultima_test(model, data_loader, name):
    model.eval()
    fin_targets=[]
    fin_outputs=[]
    with torch.no_grad():
        for _, data in enumerate(data_loader, 0):
            Stance = data['Stance'].to(device, dtype = torch.long)

            conclusion_mask = data['C_mask'].to(device, dtype = torch.long)
            conclusion_ids = data['C_ids'].to(device, dtype = torch.long)
            conclusion_token_type_ids = data['C_token_type_ids'].to(device, dtype = torch.long)

            premise_ids = data['Premise_ids'].to(device, dtype = torch.long)
            premise_mask = data['Premise_mask'].to(device, dtype = torch.long)
            premise_token_type_ids = data['Premise_token_type_ids'].to(device, dtype = torch.long)

            targets = data['targets'].to(device, dtype = torch.float)

            outputs = model(Stance,
                conclusion_ids,
                conclusion_mask,
                conclusion_token_type_ids,
                premise_ids,
                premise_mask,
                premise_token_type_ids)
            fin_targets.extend(targets.cpu().detach().numpy().tolist())
            fin_outputs.extend(torch.sigmoid(outputs).cpu().detach().numpy().tolist())
    return fin_outputs, fin_targets

Now we find the best threshold for our logits

In [37]:
def get_threshold(model, dataloader, name):
    outputs, targets = test(model, dataloader, name)
    results = {}
    for tr in np.arange(0.1, 0.9, 0.5):
        tr = round(tr, 2)
        predictions= np.array(outputs) >= tr
        f1 = f1_score(targets, predictions, average='macro', zero_division=1)
        results[tr] = f1

    return max(results, key=results.get), results, outputs, targets

In [40]:
def ultima_get_threshold(model, dataloader, name):
    outputs, targets = ultima_test(model, dataloader, name)
    results = {}
    for tr in np.arange(0.1, 0.9, 0.5):
        tr = round(tr, 2)
        predictions= np.array(outputs) >= tr
        f1 = f1_score(targets, predictions, average='macro', zero_division=1)
        results[tr] = f1

    return max(results, key=results.get), results, outputs, targets

BERT w/C

In [36]:
total_steps = len(training_loader)*EPOCHS

optimizer = torch.optim.Adam(params=C_model.parameters(), lr=LEARNING_RATE)
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=int(0.1 * total_steps), num_training_steps=total_steps)

In [39]:
name="C"

val_loss_min = np.Inf
for epoch in range(EPOCHS):
  trainBert(C_model,
            training_loader,
            optimizer,
            scheduler,
            LEARNING_RATE,
            EPOCHS,
            epoch,
            name)
  val_loss_min = validation(epoch,
                            val_loss_min,
                            CPS_model,
                            validation_loader,
                            name)

Epoch: 0, Loss:  0.5694283246994019

Train loss: 0.569428  [    0/ 5393]
Train loss: 0.615106  [  800/ 5393]
Train loss: 0.586849  [ 1600/ 5393]
Train loss: 0.763338  [ 2400/ 5393]
Train loss: 0.634926  [ 3200/ 5393]
Train loss: 0.595453  [ 4000/ 5393]
Train loss: 0.580776  [ 4800/ 5393]
0.6957143867066138

Validation loss: 0.695714.
Saving model ...
Saved PyTorch Model State to model.pth

Epoch: 1, Loss:  0.4876249432563782

Train loss: 0.487625  [    0/ 5393]
Train loss: 0.582788  [  800/ 5393]
Train loss: 0.465586  [ 1600/ 5393]
Train loss: 0.817537  [ 2400/ 5393]
Train loss: 0.619306  [ 3200/ 5393]
Train loss: 0.597615  [ 4000/ 5393]
Train loss: 0.561230  [ 4800/ 5393]
0.695714386203621

Validation loss: 0.695714.
Validation loss decreased (0.69571439 --> 0.69571439).  Saving model ...
Saved PyTorch Model State to model.pth

Epoch: 2, Loss:  0.4272211194038391

Train loss: 0.427221  [    0/ 5393]
Train loss: 0.582931  [  800/ 5393]
Train loss: 0.431963  [ 1600/ 5393]
Train loss: 0.

In [40]:
th, results_c, outputs_c, targets_c= get_threshold(C_model, validation_loader, name)
print(f"Best threshold: {th}, F1-score: {results_c[th]}")
predictions_c = np.array(outputs_c) >= th
report_c = classification_report(targets_c, predictions_c, target_names=labels ,zero_division=1)
print(report_c)

Best threshold: 0.1, F1-score: 0.73519230175329
                    precision    recall  f1-score   support

Openness to change       0.37      1.00      0.54       698
      Conservation       0.49      1.00      0.66       924
  Self-enhancement       0.75      1.00      0.86      1426
Self-transcendence       0.79      1.00      0.89      1506

         micro avg       0.60      1.00      0.75      4554
         macro avg       0.60      1.00      0.74      4554
      weighted avg       0.65      1.00      0.78      4554
       samples avg       0.60      1.00      0.73      4554



BERT w/ CP

In [41]:
total_steps = len(training_loader)*EPOCHS

optimizer = torch.optim.Adam(params=CP_model.parameters(), lr=LEARNING_RATE)
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=int(0.1 * total_steps), num_training_steps=total_steps)

In [42]:
name="CP"

val_loss_min = np.Inf
for epoch in range(EPOCHS):
  trainBert(CP_model,
            training_loader,
            optimizer,
            scheduler,
            LEARNING_RATE,
            EPOCHS,
            epoch,
            name)
  val_loss_min = validation(epoch,
                            val_loss_min,
                            CPS_model,
                            validation_loader,
                            name)

Epoch: 0, Loss:  0.6785759925842285

Train loss: 0.678576  [    0/ 5393]
Train loss: 0.654533  [  800/ 5393]
Train loss: 0.606955  [ 1600/ 5393]
Train loss: 0.701689  [ 2400/ 5393]
Train loss: 0.703128  [ 3200/ 5393]
Train loss: 0.612610  [ 4000/ 5393]
Train loss: 0.520943  [ 4800/ 5393]
0.6958106497169044

Validation loss: 0.695811.
Saving model ...
Saved PyTorch Model State to model.pth

Epoch: 1, Loss:  0.4321841895580292

Train loss: 0.432184  [    0/ 5393]
Train loss: 0.553686  [  800/ 5393]
Train loss: 0.413644  [ 1600/ 5393]
Train loss: 0.609383  [ 2400/ 5393]
Train loss: 0.702806  [ 3200/ 5393]
Train loss: 0.560953  [ 4000/ 5393]
Train loss: 0.467353  [ 4800/ 5393]
0.6958106497169044

Validation loss: 0.695811.
Validation loss decreased (0.69581065 --> 0.69581065).  Saving model ...
Saved PyTorch Model State to model.pth

Epoch: 2, Loss:  0.35810524225234985

Train loss: 0.358105  [    0/ 5393]
Train loss: 0.475511  [  800/ 5393]
Train loss: 0.366076  [ 1600/ 5393]
Train loss: 

In [43]:
th, results_cp, outputs_cp, targets_cp= get_threshold(CP_model, validation_loader, name)
print(f"Best threshold: {th}, F1-score: {results_cp[th]}")
predictions_cp = np.array(outputs_cp) >= th
report_cp = classification_report(targets_cp, predictions_cp, target_names=labels ,zero_division=1)
print(report_cp)

Best threshold: 0.1, F1-score: 0.7525513395108098
                    precision    recall  f1-score   support

Openness to change       0.46      0.92      0.61       698
      Conservation       0.49      1.00      0.66       924
  Self-enhancement       0.75      1.00      0.86      1426
Self-transcendence       0.79      1.00      0.89      1506

         micro avg       0.63      0.99      0.77      4554
         macro avg       0.62      0.98      0.75      4554
      weighted avg       0.67      0.99      0.79      4554
       samples avg       0.64      0.99      0.75      4554



BERT w/ CPS

In [44]:
total_steps = len(training_loader)*EPOCHS

optimizer = torch.optim.Adam(params=CPS_model.parameters(), lr=LEARNING_RATE)
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=int(0.1 * total_steps), num_training_steps=total_steps)

In [45]:
name="CPS"

val_loss_min = np.Inf
for epoch in range(EPOCHS):
  trainBert(CPS_model,
            training_loader,
            optimizer,
            scheduler,
            LEARNING_RATE,
            EPOCHS,
            epoch,
            name)
  val_loss_min = validation(epoch,
                            val_loss_min,
                            CPS_model,
                            validation_loader,
                            name)

Epoch: 0, Loss:  0.705580472946167

Train loss: 0.705580  [    0/ 5393]
Train loss: 0.651876  [  800/ 5393]
Train loss: 0.605770  [ 1600/ 5393]
Train loss: 0.732141  [ 2400/ 5393]
Train loss: 0.630902  [ 3200/ 5393]
Train loss: 0.580477  [ 4000/ 5393]
Train loss: 0.505009  [ 4800/ 5393]
0.5478337931482098

Validation loss: 0.547834.
Saving model ...
Saved PyTorch Model State to model.pth

Epoch: 1, Loss:  0.45350003242492676

Train loss: 0.453500  [    0/ 5393]
Train loss: 0.582348  [  800/ 5393]
Train loss: 0.404408  [ 1600/ 5393]
Train loss: 0.631282  [ 2400/ 5393]
Train loss: 0.716967  [ 3200/ 5393]
Train loss: 0.577514  [ 4000/ 5393]
Train loss: 0.441378  [ 4800/ 5393]
0.5326776671761702

Validation loss: 0.532678.
Validation loss decreased (0.54783379 --> 0.53267767).  Saving model ...
Saved PyTorch Model State to model.pth

Epoch: 2, Loss:  0.4007147550582886

Train loss: 0.400715  [    0/ 5393]
Train loss: 0.501300  [  800/ 5393]
Train loss: 0.313188  [ 1600/ 5393]
Train loss: 0

In [46]:
th, results_cps, outputs_cps, targets_cps= get_threshold(CPS_model, validation_loader, name)
print(f"Best threshold: {th}, F1-score: {results_cps[th]}")
predictions_cps = np.array(outputs_cps) >= th
report_cps = classification_report(targets_cps, predictions_cps, target_names=labels ,zero_division=1)
print(report_cps)

Best threshold: 0.1, F1-score: 0.7505227638675923
                    precision    recall  f1-score   support

Openness to change       0.45      0.90      0.60       698
      Conservation       0.49      0.99      0.66       924
  Self-enhancement       0.75      1.00      0.86      1426
Self-transcendence       0.79      1.00      0.89      1506

         micro avg       0.63      0.98      0.77      4554
         macro avg       0.62      0.97      0.75      4554
      weighted avg       0.67      0.98      0.79      4554
       samples avg       0.64      0.99      0.75      4554



Ultimate model:

In [33]:
total_steps = len(training_loader)*EPOCHS

optimizer = torch.optim.Adam(params=ultima_model.parameters(), lr=LEARNING_RATE)
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=int(0.1 * total_steps), num_training_steps=total_steps)

In [34]:
name="ultima"

val_loss_min = np.Inf
for epoch in range(EPOCHS):
  ultima_train(ultima_model,
            training_loader,
            optimizer,
            scheduler,
            LEARNING_RATE,
            EPOCHS,
            epoch)
  val_loss_min = ultima_validation(epoch,
                            val_loss_min,
                            ultima_model,
                            validation_loader,
                            name)

Epoch: 0, Loss:  0.6953879594802856

Train loss: 0.695388  [    0/ 5393]
Train loss: 0.638127  [  800/ 5393]
Train loss: 0.608856  [ 1600/ 5393]
Train loss: 0.717975  [ 2400/ 5393]
Train loss: 0.714563  [ 3200/ 5393]
Train loss: 0.578909  [ 4000/ 5393]
Train loss: 0.521495  [ 4800/ 5393]
0.551617231550096

Validation loss: 0.551617.
Saving model ...
Saved PyTorch Model State to model.pth

Epoch: 1, Loss:  0.44719764590263367

Train loss: 0.447198  [    0/ 5393]
Train loss: 0.533948  [  800/ 5393]
Train loss: 0.472336  [ 1600/ 5393]
Train loss: 0.672140  [ 2400/ 5393]
Train loss: 0.577875  [ 3200/ 5393]
Train loss: 0.490906  [ 4000/ 5393]
Train loss: 0.509997  [ 4800/ 5393]
0.5261396324332757

Validation loss: 0.526140.
Validation loss decreased (0.55161723 --> 0.52613963).  Saving model ...
Saved PyTorch Model State to model.pth

Epoch: 2, Loss:  0.38626694679260254

Train loss: 0.386267  [    0/ 5393]
Train loss: 0.482667  [  800/ 5393]
Train loss: 0.394080  [ 1600/ 5393]
Train loss: 

In [42]:
th, results_ultima, outputs_ultima, targets_ultima= ultima_get_threshold(ultima_model, validation_loader, name)
print(f"Best threshold: {th}, F1-score: {results_ultima[th]}")
predictions_ultima = np.array(outputs_ultima) >= th
report_roberta_ultima = classification_report(targets_ultima, predictions_ultima, target_names=labels ,zero_division=1)
print(report_roberta_ultima)

Best threshold: 0.1, F1-score: 0.7482409466365485
                    precision    recall  f1-score   support

Openness to change       0.43      0.94      0.59       698
      Conservation       0.49      1.00      0.66       924
  Self-enhancement       0.75      1.00      0.86      1426
Self-transcendence       0.79      1.00      0.89      1506

         micro avg       0.63      0.99      0.77      4554
         macro avg       0.62      0.99      0.75      4554
      weighted avg       0.66      0.99      0.79      4554
       samples avg       0.63      0.99      0.75      4554



One important fact is that since the main goal is getting a good F1 score not a high accuracy; we believe we could've used a custom loss function to further improve the results.